In [1]:
import pandas as pd

In [2]:
def saveDataFrame(dataframes,filepath,names):
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    for df,sname in zip(dataframes,names):
        df.to_excel(writer,sheet_name=sname)   
    writer.save()

In [3]:
"""processing related codes"""
def initializeFrames(df):
    for col in df.columns:
        if col !='IPCNo':
            df[col].values[:] = 0
    return df
"""Calculate Distribution Factor"""
def calcualteTotalBill(df):
    for index,row in df.iterrows():
        dtotal=0
        for i in range(2,13):
            dtotal+=df.iloc[index,i]
            
        #total=row['Gob']+row['Rpa']
        #print(total)
        df.iloc[index,13]=dtotal
    return df
"""Calcualte Distribution Factor"""
def calcualteTotalBill2(df):
    for index,row in df.iterrows():
        total=row['Gob']+row['Rpa']
        #print(total)
        df.iloc[index,13]=total
    return df
def calcualteDistributionFactor(df1,df2):
    for index,row in df1.iterrows():
        #print(row['id'])
        total=row['Total']
        dtotal=0
        for i in range(2,13):
            dv=df1.iloc[index,i]/total
            df2.iloc[index,i]=dv
            dtotal+=dv
        df2.iloc[index,13]=dtotal
    return df2
def calculateDistributedValueGob(df1,df2,df3):
    for index,row in df1.iterrows():
        dtotal=0
        total=row['Gob']
        for i in range(2,13):
            distributed_val=round(df2.iloc[index,i]*total,2)
            dtotal+=distributed_val
            df3.iloc[index,i]=distributed_val
        df3.iloc[index,13]=dtotal
    return df3
def calculateDistributedValueGob_Mob(df1,df2,df3):
    package_list=list(df2['Package_No'])
    print(package_list)
    for index,row in df1.iterrows():
        dtotal=0
        total=row['Gob']
        pno=row['Package No'].strip()
        #pno=df1['Package No']
        print("package no={} mobilization_gob={}".format(pno,total))
        dist_df_row=package_list.index(pno)        
        for i in range(2,13):
            dfactor=df2.iloc[dist_df_row,i]
            #print("package no={}index={} factor={}".format(pno,dist_df_row,dfactor))
            distributed_val=round(dfactor*total,2)
            #dtotal+=distributed_val
            df3.iloc[index,i]=distributed_val
        #df3.iloc[index,13]=dtotal
    return df3
def calculateDistributedValueRpa(df1,df2,df3):
    for index,row in df1.iterrows():
        dtotal=0
        total=row['Rpa']        
        for i in range(2,13):
            distributed_val=round(df2.iloc[index,i]*total,2)
            dtotal+=distributed_val
            df3.iloc[index,i]=distributed_val
        df3.iloc[index,13]=dtotal
    return df3
def calculateDistributedValueRpa_Mob(df1,df2,df3):
    package_list=list(df2['Package_No'])
    print(package_list)
    for index,row in df1.iterrows():
        dtotal=0
        total=row['Rpa']
        pno=row['Package No'].strip()
        #pno=df1['Package No']
        print("package no={} mobilization_rpa={}".format(pno,total))
        #print("package no={}".format(pno))
        dist_df_row=package_list.index(pno)        
        for i in range(2,13):
            dfactor=df2.iloc[dist_df_row,i]
            #print("package no={}index={} factor={}".format(pno,dist_df_row,dfactor))
            distributed_val=round(dfactor*total,2)
            #dtotal+=distributed_val
            df3.iloc[index,i]=distributed_val
        #df3.iloc[index,13]=dtotal
    return df3
    
def getTotal(df,index):
    dtotal=0
    for i in range(2,13):
        dtotal +=df.iloc[index,i]
    return dtotal
def packageWiseCost(df1,df2,df3):
    package_list=list(df2['PackageNo'])
    for index,row in df1.iterrows():
        pno=row['Package No'].strip()
        r=package_list.index(pno)
        c=int(row['IPC No'])
        gob_val=getTotal(df3,index)
        #gob_val=df3.iloc[index,13]
        df2.iloc[r,c]= df2.iloc[r,c]+gob_val
        print("index={} packge={} row={} column={}".format(index,pno,r,c))
    return df2
def getPacakgeWiseTotal(df):
    for index,row in df.iterrows():
        dtotal=0
        for i in range(1,21):
            dtotal +=df.iloc[index,i]
        df.iloc[index,21]=dtotal
    return df
def combineMobilizationWithActutal(df1,df2):
    df3=df1.copy(deep=True)
    col_list=list(df1.columns)
    print(col_list)
    for i in range(2,12):
        colname=col_list[i]
        mob_val=list(df1[colname])
        str_val=list(df2[colname])
        com_list=[mob_val[i]+str_val[i] for i in range(len(mob_val))]
        df3[colname]=com_list
        #print(com_list)
    for index,row in df3.iterrows():
        total=0
        for i in range(2,13):
            total=total+df3.iloc[index,i]
        df3.iloc[index,13]=total
    return df3
    
   

In [4]:
input_path=r'E:\Website_24_11_2020\cmis6\cmis6\Civilworks cost\IPC Distribution.xlsx' #home computer
output_path=r'E:\Website_24_11_2020\cmis6\cmis6\Civilworks cost\output.xlsx' #home computer
#input_path=r'F:\website\cmis6\Civilworks cost\IPC Distribution.xlsx' #office  computer
#output_path=r'F:\website\cmis6\Civilworks cost\output.xlsx' #office  computer

In [5]:
sheetName='IPC_Dist'
mydf=pd.read_excel(input_path,sheet_name=sheetName)
mydf.fillna(0,inplace=True)
dist_df=mydf.copy(deep=True)
rpa_df=mydf.copy(deep=True)
gob_df=mydf.copy(deep=True)
dist_df=dist_df.drop(['Gob','Rpa'],axis=1)
rpa_df=rpa_df.drop(['Gob','Rpa'],axis=1)
gob_df=gob_df.drop(['Gob','Rpa'],axis=1)


In [6]:
"""Initializing frames"""
dist_df=initializeFrames(dist_df)
rpa_df=initializeFrames(rpa_df)
gob_df=initializeFrames(gob_df)
mydf=calcualteTotalBill(mydf)
dist_df=calcualteDistributionFactor(mydf,dist_df)
gob_df=calculateDistributedValueGob(mydf,dist_df,gob_df)
rpa_df=calculateDistributedValueRpa(mydf,dist_df,rpa_df)

In [7]:
"""calcualting packagewise cost"""
sheetName='Package_wise_cost'
pac_cost_gob_df=pd.read_excel(input_path,sheet_name=sheetName)
pac_cost_gob_df.fillna(0,inplace=True)
pac_cost_rpa_df=pac_cost_gob_df.copy(deep=True)
pac_cost_gob_df=packageWiseCost(mydf,pac_cost_gob_df,gob_df)
pac_cost_gob_df=getPacakgeWiseTotal(pac_cost_gob_df)
pac_cost_rpa_df=packageWiseCost(mydf,pac_cost_rpa_df,rpa_df)
pac_cost_rpa_df=getPacakgeWiseTotal(pac_cost_rpa_df)

index=0 packge=HOBI-06 row=34 column=3
index=1 packge=KISH-02 row=1 column=5
index=2 packge=KISH-09 row=7 column=2
index=3 packge=KISH-18 row=16 column=3
index=4 packge=KISH-17 row=15 column=5
index=5 packge=KISH-12 row=10 column=2
index=6 packge=HOBI-02 row=31 column=1
index=7 packge=KISH-18 row=16 column=4
index=8 packge=KISH-25 row=23 column=1
index=9 packge=SUNM-04 row=48 column=1
index=10 packge=KISH-06 row=5 column=7
index=11 packge=HOBI-01 row=30 column=1
index=12 packge=KISH-06 row=5 column=4
index=13 packge=KISH-16 row=14 column=6
index=14 packge=KISH-19 row=17 column=2
index=15 packge=NETR-02 row=38 column=1
index=16 packge=HOBI-02 row=31 column=2
index=17 packge=HOBI-04 row=32 column=3
index=18 packge=KISH-15 row=13 column=3
index=19 packge=SUNM-04 row=48 column=2
index=20 packge=SUNM-06 row=50 column=1
index=21 packge=KISH-18 row=16 column=5
index=22 packge=KISH-12 row=10 column=3
index=23 packge=SUNM-01 row=45 column=1
index=24 packge=KISH-20 row=18 column=2
index=25 packg

In [8]:
sheetName='Mobilization_Dist'
mobilization_df=pd.read_excel(input_path,sheet_name=sheetName)
mobilization_df.fillna(0,inplace=True)
rpa_df1=rpa_df.copy(deep=True)
gob_df1=gob_df.copy(deep=True)
sheetName='Package_wise_str_df'
mobilization_distribution_df=pd.read_excel(input_path,sheet_name=sheetName)
mobilization_distribution_df
mobilization_gob_df=initializeFrames(gob_df1)
mobilization_rpa_df=initializeFrames(rpa_df1)
mobilization_gob_df
mobilization_rpa_df
mobilization_gob_df=calculateDistributedValueGob_Mob(mobilization_df,mobilization_distribution_df,mobilization_gob_df)
mobilization_rpa_df=calculateDistributedValueRpa_Mob(mobilization_df,mobilization_distribution_df,mobilization_rpa_df)
comb_gob=combineMobilizationWithActutal(mobilization_gob_df,gob_df)
com_rpa=combineMobilizationWithActutal(mobilization_rpa_df,rpa_df)

['KISH-01', 'KISH-02', 'KISH-03', 'KISH-04', 'KISH-05', 'KISH-06', 'KISH-07', 'KISH-09', 'KISH-10', 'KISH-11', 'KISH-12', 'KISH-13', 'KISH-14', 'KISH-15', 'KISH-16', 'KISH-17', 'KISH-18', 'KISH-19', 'KISH-20', 'KISH-21', 'KISH-22', 'KISH-23', 'KISH-24', 'KISH-25', 'KISH-26', 'KISH-27', 'NETR-01', 'NETR-02', 'NETR-03', 'NETR-04', 'NETR-05', 'NETR-06', 'NETR-07', 'NETR-08', 'SUNM-01', 'SUNM-02', 'SUNM-03', 'SUNM-04', 'SUNM-05', 'SUNM-06', 'HOBI-01', 'HOBI-02', 'HOBI-04', 'HOBI-05', 'HOBI-06', 'HOBI-07', 'KISH-28/Lot-1', 'KISH-28/Lot-2', 'KISH-28/Lot-3', 'KISH-28/Lot-4']
package no=HOBI-06 mobilization_gob=0.0
package no=KISH-02 mobilization_gob=17597.86
package no=KISH-09 mobilization_gob=0.0
package no=KISH-18 mobilization_gob=0.0
package no=KISH-17 mobilization_gob=0.0
package no=KISH-12 mobilization_gob=0.0
package no=HOBI-02 mobilization_gob=21000.000000000004
package no=KISH-18 mobilization_gob=0.0
package no=KISH-25 mobilization_gob=0.0
package no=SUNM-04 mobilization_gob=129511.50

['Sl No ', 'IPCNo', 'Inlet', 'Reg_Rehab_Rehab', 'Reg_CW_Box', 'Khal_Riv_New', 'Khal_Riv_Rehab', 'Full_Emb_Rehab', 'Sub_Emb_Rehab', 'Sub_Emb_Const', 'Reg_Rehab_New', 'Wmg_Office', 'Gate_Repair', 'Total', 'Package No', 'IPC No', 'Month']


In [9]:
frames=[]
names=[]
frames.append(mydf)
frames.append(dist_df)
frames.append(rpa_df)
frames.append(gob_df)
frames.append(pac_cost_gob_df)
frames.append(pac_cost_rpa_df)
names.append('original')
names.append('dfactor')
names.append("rpa")
names.append("gob")
names.append("package_wise_gob")
names.append("package_wise_rpa")
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
frames.append(mobilization_gob_df)
names.append("mobilization_gob")
frames.append(mobilization_rpa_df)
names.append("mobilization_rpa")
frames.append(comb_gob)
names.append("gob_combined")
frames.append(com_rpa)
names.append("rpa_combined")
frames.append(mobilization_df)
names.append("mobilization_original_input")
saveDataFrame(frames,output_path,names)

In [10]:
mobilization_df

,Sl No,IPCNo,Inlet,Reg_Rehab_Rehab,Reg_CW_Box,Khal_Riv_New,Khal_Riv_Rehab,Full_Emb_Rehab,Sub_Emb_Rehab,Sub_Emb_Const,Reg_Rehab_New,Wmg_Office,Gate_Repair,Total,Gob,Rpa,Package No,IPC No,Month
0,1,75/hobi-06 3rd Bill,0.000,0.0,2821414.000,0.00,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.00,0.0000,0.0000,HOBI-06,3,10
1,2,124/kish-02 5th Bill,0.000,0.0,788965.620,0.00,0.0,0.0,0.00,855983.510,0.0,0.0,0.0,125699.00,17597.8600,108101.1400,KISH-02,5,12
2,3,142/kish-09 2nd Bill,0.000,0.0,7908184.000,0.00,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.00,0.0000,0.0000,KISH-09,2,12
3,4,143/kish-18 3rd Bill,0.000,0.0,3391138.500,8657627.44,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.00,0.0000,0.0000,KISH-18,3,12
4,5,149/kish-17 5th Bill,654830.495,0.0,1764444.173,0.00,0.0,0.0,0.00,5957470.400,0.0,0.0,0.0,0.00,0.0000,0.0000,KISH-17,5,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,92,473/kish-16 8th Bill,0.000,0.0,120705.260,0.00,0.0,0.0,0.00,116300.225,0.0,0.0,0.0,0.00,0.0000,0.0000,KISH-16,8,6
88,93,469/kish-19 5th Bill,0.000,0.0,0.000,7869228.00,0.0,0.0,0.00,0.000,0.0,0.0,0.0,483525.43,67693.5602,415831.8698,KISH-19,5,6
89,94,472/kish-17 8th Bill,795781.300,0.0,1039759.708,0.00,0.0,0.0,0.00,5168075.669,0.0,0.0,0.0,0.00,0.0000,0.0000,KISH-17,8,6
90,77,425/hobi-04 4th Bill,0.000,0.0,0.000,0.00,0.0,0.0,4828643.00,0.000,0.0,0.0,0.0,486119.06,68056.6684,418062.3916,HOBI-04,4,6
